In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df = pd.read_csv('/kaggle/input/5-million-song-lyrics-dataset/ds2.csv',
                nrows = 500)[['title', 'artist','lyrics']]
#df

In [ ]:
sortedcol = (df.set_index(['title', 'artist'])
   .apply(lambda x: x.str.split('\n').explode())
   .reset_index())

sortedcol["textorder"] = sortedcol.groupby('title')['artist'].rank(method="first", ascending=True).astype(np.int64)
sortedcol["must_remove"] = sortedcol.lyrics.map(lambda x: 1 if ('[' in x)|(x == '') else 0)

sortedcol2 = sortedcol[sortedcol.must_remove == 0]

In [ ]:
sortedcol3 = sortedcol2.groupby(['title','artist','lyrics']).max().reset_index()
sortedcol4 = sortedcol3.sort_values(['artist','title','textorder'])
sortedcol5 = sortedcol4.drop_duplicates()
sortedcol5["text_order"] = sortedcol5.groupby('title')['artist'].rank(method="first", ascending=True).astype(np.int64)
finaldata = sortedcol5[['title', 'artist','lyrics', 'text_order']]
finaldata.to_csv('musicclean.csv',index=False)